# CS229: PS1
## Problem 5


In [1]:
%matplotlib inline
import numpy as np 
import scipy as sp 
import matplotlib as mpl
import matplotlib.cm as cm 
import matplotlib.pyplot as plt 
import pandas as pd 
import seaborn as sns 

pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)

sns.set_context('notebook')

In [ ]:
df_train = pd.read_csv('./data/quasar_train.csv', )
cols_train = df_train.columns.values.astype(float).astype(int)
df_train.shape
df_train.head()

In [ ]:
df_test = pd.read_csv('./data/quasar_test.csv', )
cols_test = df_train.columns.values.astype(float).astype(int)

assert(cols_train == cols_test).all()

df_train.columns = cols_train
df_test.columns = cols_test
df_test.head()

In [4]:
def normal_equation(x, y, w=None):
    if w is None:
        return np.linalg.inv(x.T.dot(x)).dot(x.T).dot(y)
    else:
        return np.linalg.inv(x.T.dot(w).dot(x)).dot(x.T).dot(w).dot(y)
    

In [5]:
def build_weights(x, x_i, tau=5):
    return np.diag(np.exp(-((x-x_i)[:,1]**2)/(2*tau**2)))

In [6]:
y=df_train.head(1).values.T
x=np.vstack((np.ones(cols_train.shape), cols_train)).T

In [ ]:
theta = normal_equation(x,y)
print(theta.ravel())

In [ ]:
ax = sns.regplot(x=x[:,1], y=y, fit_reg=False);
plt.plot(x[:,1], x.dot(theta), linewidth=5);
ax.set(xlabel="Wavelength", ylabel='Flux');

In [9]:
pred = []
for k, x_j in enumerate(x):
    w = build_weights(x, x_j,5)
    theta = normal_equation(x, y, w)
    pred.append(theta.T.dot(x_j[:,np.newaxis]).ravel()[0])

In [ ]:
ax = sns.regplot(x=x[:,1], y=y, fit_reg=False);
plt.plot(x[:,1],pred,linewidth=3);
ax.set(xlabel="Wavelength", ylabel='Flux');

In [ ]:
fig,axes = plt.subplots(2, 2, figsize=(14,10))
axes = axes.ravel()

colors = sns.color_palette("muted")

for k, tau in enumerate(np.logspace(0,3,4).astype(int)):
    pred = []
    ax=axes[k]
    for x_j in x:
        w = build_weights(x, x_j,tau)
        theta = normal_equation(x, y, w)
        pred.append(theta.T.dot(x_j[:,np.newaxis]).ravel()[0])

    sns.regplot(x=x[:,1], y=y, fit_reg=False, ax=ax, color=colors[0]);
    ax.plot(x[:,1],pred,linewidth=3,color=colors[k+1]);
    ax.set(xlabel="Wavelength", ylabel='Flux', title='tau = {}'.format(tau));

In [12]:
y_train=df_train.values.T
y_test=df_test.values.T

In [13]:
def smoother(x,y_in,tau):
    pred = np.zeros(y_in.shape)
    for i in range(y_in.shape[1]):
        y = y_in[:,i]
        for j, x_j in enumerate(x):
            w = build_weights(x, x_j,tau)
            theta = normal_equation(x, y, w)
            pred[j][i] =  theta.T.dot(x_j[:,np.newaxis]).ravel()[0]
    return pred

In [14]:
y_train_smooth = smoother(x,y_train,5)
y_test_smooth = smoother(x,y_test,5)

In [15]:
def compute_delta(y,y_i):
    return np.sum((y-y_i[:,np.newaxis])**2,0)

In [16]:
def ker(t):
    return np.maximum(1-t,0)

In [17]:
def get_nearest_neighbors(y, i, deltas, k):
    max_d = deltas.max()
    idx = np.argsort(deltas)[:k+1]
    return idx[:k]

In [18]:
def func_regression(x, y_train, y_test, lyman_alpha):
    y_train_left = y_train[x < lyman_alpha,:]
    y_train_right = y_train[x >= lyman_alpha+100,:]
    y_test_left = y_test[x < lyman_alpha,:]
    y_test_right = y_test[x >= lyman_alpha+100,:]

    f_hat_left = np.zeros(y_test_left.shape)

    for i in range(y_test_right.shape[1]):
        deltas = compute_delta(y_train_right, y_test_right[:,i])
        idx = get_nearest_neighbors(y_train_right,i,deltas,3)
        h = np.max(deltas) 
        weights = ker(deltas/h)[idx]
        
        f_hat_num = np.sum(y_train_left[:,idx]*weights,1)
        f_hat_den = np.sum(weights)
        f_hat = f_hat_num/f_hat_den
        f_hat_left[:,i] = f_hat
    return f_hat_left

In [19]:
f_hat_train = func_regression(x[:,1],y_train_smooth,y_train_smooth,1200)

In [ ]:
y_train_left = y_train_smooth[x[:,1] < 1200,:]
y_test_left = y_test_smooth[x[:,1] < 1200,:]

ax = plt.plot(x[x[:,1] < 1200,1],f_hat_train);
h = np.mean(np.sum((f_hat_train-y_train_left)**2,0));
print(h)

In [22]:
f_hat_test = func_regression(x[:,1],y_train_smooth,y_test_smooth,1200)

In [ ]:
ax = plt.plot(x[x[:,1] < 1200,1],f_hat_test);
h = np.mean(np.sum((f_hat_test-y_test_left)**2,0))
print(h)

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(16,6))
axes[0].plot(x[x[:,1] < 1200,1],f_hat_test[:,0]);
axes[0].plot(x[x[:,1] < 1200,1],y_test_left[:,0]);
axes[0].set(xlabel='Wavelength', ylabel='Flux', title='Training Ex1');
axes[0].legend(['Estimate','Smoothed Value']);

axes[1].plot(x[x[:,1] < 1200,1],f_hat_test[:,5]);
axes[1].plot(x[x[:,1] < 1200,1],y_test_left[:,5]);
axes[1].set(xlabel='Wavelength', ylabel='Flux', title='Training Ex6');
axes[1].legend(['Estimate','Smoothed Value']);